## env
**install jupyter-lab**

https://jupyter.org/install

```sh
python3 -m pip install jupyterlab notebook
```


**install tslab, a typescript kernel for jupyter**

https://github.com/yunabe/tslab

```sh
npm install -g tslab
tslab install python=python3
```

## usage

3 ways:

**connect to remote server**

- run jupyter 

```sh
    jupyter-lab --no-browser --NotebookApp.allow_origin='*'
```

```text
    [C 15:47:28.854 NotebookApp]
        To access the notebook, open this file in a browser:
            file:///Users/drincanngao/Library/Jupyter/runtime/nbserver-8178-open.html
        Or copy and paste one of these URLs:
            http://localhost:8888/?token=2fd985f93b6f39d1dfaecc334c0b425ef3430008f740a666
        or http://127.0.0.1:8888/?token=2fd985f93b6f39d1dfaecc334c0b425ef3430008f740a666
```

- install `Jupyter` extension in vscode and open this file.

- put the url '127.0.0.1' one into vscode "select another kernel" > "existing jupyter server" > "input connection url"

**start ts kernel in vscode**:

- "select another kernel" > "jupyter kernel" > "tslab"

**browser**

- open the directory this file locates in terminal and run

```sh
    jupyter-lab
```

- the browser will open automatically



## nodejs event loop
see -> https://nodejs.org/en/docs/guides/event-loop-timers-and-nexttick

```text
   ┌───────────────────────────┐
┌─>│           timers          │
│  └─────────────┬─────────────┘
│  ┌─────────────┴─────────────┐
│  │     pending callbacks     │
│  └─────────────┬─────────────┘
│  ┌─────────────┴─────────────┐
│  │       idle, prepare       │
│  └─────────────┬─────────────┘      ┌───────────────┐
│  ┌─────────────┴─────────────┐      │   incoming:   │
│  │           poll            │<─────┤  connections, │
│  └─────────────┬─────────────┘      │   data, etc.  │
│  ┌─────────────┴─────────────┐      └───────────────┘
│  │           check           │
│  └─────────────┬─────────────┘
│  ┌─────────────┴─────────────┐
└──┤      close callbacks      │
   └───────────────────────────┘
```

## the article of this notebook

[[手写协程] 从事件循环到 Promise/Thunk](https://codingfor.life/#/%E6%8A%80%E6%9C%AF%E6%96%87%E7%AB%A0/%E5%8D%8F%E7%A8%8B%E7%9A%84%E6%8A%BD%E8%B1%A1%E5%92%8C%E5%AE%9E%E7%8E%B0.md)

#### 支持协程的部分语言
- stackless
  - python(std lib: asyncio) https://docs.python.org/3/library/asyncio.html?highlight=asyncio
  - kotlin https://kotlinlang.org/docs/coroutines-overview.html
  - cpp(std lib) https://en.cppreference.com/w/cpp/language/coroutines
  - rust
  - javascript(built-in) https://developer.mozilla.org/en-US/docs/Web/JavaScript/Event_loop
    - browser
    - nodejs https://nodejs.org/en/docs/guides/event-loop-timers-and-nexttick
- stackful
  - golang
  - java(java 19: virtual threads) https://openjdk.org/jeps/444


#### 无栈协程的核心特性
- 在用户态实现 (非抢占)
- 非阻塞 I/O (单线程并发)
- 上下文不关心调用栈 (广度优先调度)

#### 实现参考
- 协程的抽象 JavaScript 实现 https://github.com/Drincann/js-coroutine/
- 协程的抽象 + 协程调度器 python 实现 https://github.com/Drincann/py-coroutine/

#### 下层为上层提供的异步 api:

In [ ]:
import { readFile } from 'fs';
readFile('/etc/hosts', (err, data) => {
  console.log(1);
});
console.log(2);

// output 2 1

#### 上层协程的抽象需要使用以下几个工具
- 生成器
- Promise

JavaScript 异步控制方案演变：

阶段 1 回调函数

In [ ]:
import fs, { readFile } from 'fs'

// 阶段 1
fs.readFile('./example.txt', (err, data1) => {

  const path = data1.toString()
  fs.readFile(path, (err, hostsContent) => {
    console.log(hostsContent?.toString())
  })

})

阶段 2 Promise

In [ ]:
import fs, { readFile } from 'fs'

new Promise<ArrayBuffer>(resolve =>{

  readFile('./example.txt', (err, data1) => {
    resolve(data1)
  })

}).then(data1 => {

  const path = data1.toString()
  return new Promise(resolve => {
    readFile(path, (err, hostsContent) => {
      resolve(hostsContent)
    })
  })

}).then(hostsContent => {

  console.log(hostsContent?.toString())

})

阶段 3 async / await

In [ ]:
import fs, { readFile } from 'fs'

// 阶段 3
let readFilePromises = (filename: string) => {
  return new Promise<ArrayBuffer>(resolve =>{
    readFile(filename, (err, data1) => {
      resolve(data1)
    })
  })
}

void async function () {
  const path = await readFilePromises('./example.txt')
  let hostsContent = await readFilePromises(path.toString())
  console.log(hostsContent?.toString())
}()

async function 几乎是 Promise 和生成器函数的语法糖，在解释器中与生成器共同一套实现，他们非常相似：
```ts
async function() {
  const re1 = await foo();
  const re2 = await bar(re1);
  return [re1, re2]
}

function*() {
  const re1 = yield foo();
  const re2 = yield bar(re1);
  return [re1, re2]
}
```

In [ ]:
const log = console.log.bind(console)
function* genfun() {
  let re = yield 1;
  yield re;
  console.log('done')
  return 'returnVal';
}

const gen = genfun();
log(gen.next()); // {value: 1, done: false}
log(gen.next(2)); // {value: 2, done: false}
log(gen.next()); // {value: 'returnVal', done: true}
log(gen.next()); // {value: undefined, done: true}
log(gen.next()); // {value: undefined, done: true}

只有后生成器不够，我们还需要一个执行生成器的东西：

In [ ]:
class Coroutine<T> {
  private coroutine: Generator<any, T, any>;

  public constructor(gen: Generator<any, T, any>) {
    this.coroutine = gen;
  }

  execute(): void {
    const next = (value?: any) => {
      // preform the next step
      const result = this.coroutine.next(value);
      if (!result.done) { // generator is not finished
        next(result.value); // set the return value of yield
      }
    }; 
    
    next();
  }
}

跑一下看看：

In [ ]:
function* genfun() {
  const a = yield 1;
  console.log(a);
  const b = yield 2;
  console.log(b);
}

new Coroutine(genfun()).execute(); // output: 1\n2\n

让我们的执行器可以像 async function 那样处理 Promise：

In [ ]:
class Coroutine<T> {
  private coroutine: Generator<any, T, any>;

  public constructor(gen: Generator<any, T, any>) {
    this.coroutine = gen;
  }

  execute(): void {
    const next = (value?: any) => {
      const result = this.coroutine.next(value);
      if (!result.done) {
        if (typeof result.value.then === 'function') { // thenable
          result.value.then(data => { // call next when promise is resolved
            next(data);
          });
        } else {
          next(result.value)
        }
      }
    }; next();
  }
}

我们的执行器可以同步地处理异步操作了：

In [ ]:
import fs, { readFile } from 'fs'

let readFilePromises = (filename: string) => {
  return new Promise<ArrayBuffer>(resolve =>{
    readFile(filename, (err, data1) => {
      resolve(data1)
    })
  })
}

// generator
function* genfun() {
  const fileContent = yield readFilePromises('./example.txt');
  console.log(fileContent.toString());
}

// run
new Coroutine(genfun()).execute(); // file content

还可以更 async function，我们让 execute 方法返回一个 Promise，并能处理其他生成器：

In [ ]:
function isThenable(obj: any) : obj is Promise<any> {
  return obj && typeof obj.then === 'function';
}

function isGeneratorIterator(obj: any) : obj is Generator {
  return obj && typeof obj.next === 'function';
}

class Coroutine<T> {
  private coroutine: Generator<any, T, any>;

  public constructor(gen: Generator<any, T, any>) {
    this.coroutine = gen;
  }

  execute(): Promise<T> {
    // 返回一个 promise
    return new Promise((resolve, reject) => {
      const next = (value?: any) => {
        const result = this.coroutine.next(value);
        if (result.done) { // 生成器结束时 resolve 生成器返回结果
          resolve(result.value);
        } else {
          // 处理 promise
          if (isThenable(result.value)) {
            result.value.then(next, reject);
          } else if (isGeneratorIterator(result.value)) { // 处理其他生成器
            new Coroutine(result.value).execute().then(next, reject);
          } else {
            // value
            next(result.value);
          }
        }
      }; next();
    });
  }
}

试试：

In [ ]:
import fs, { readFile } from 'fs'

let readFilePromises = (filename: string) => {
  return new Promise<ArrayBuffer>(resolve =>{
    readFile(filename, (err, data1) => {
      resolve(data1)
    })
  })
}

function* getPathFrom(filename: string) {
  return yield readFilePromises(filename);
}

function* genfun() {
  const path = yield getPathFrom('./example.txt');
  return yield readFilePromises(path.toString());
}

console.log(new Coroutine(genfun()).execute().then(data => console.log(data.toString())));

#### 参考
- 完整实现 https://github.com/Drincann/js-coroutine
- 如何实现 promise https://www.promisejs.org/implementing/

以上的逻辑实现了协程这个抽象的概念，下面来看上面的实现所依赖的异步 api 该如何实现。

```ts
readFile('/etc/hosts', (err, data) => {
  console.log(data);
});
```